In [127]:
import yaml
import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path

In [128]:


# === CONFIGURAÇÃO E PARÂMETROS DE TEMPO ===
today = datetime.today()
start_date = datetime(today.year, 1, 1)
end_date = datetime(today.year + 1, 12, 31)
date_range = pd.date_range(start=start_date, end=end_date)

In [129]:
# === CONFIGURAÇÃO DO CAMINHO DO CSV ===
csv_path = "../outputs/itau_itau_extrato_012025.csv"
df1 = pd.read_csv(csv_path, sep="|")
# Filtra df1 para remover linhas cujo texto da descrição contenha "SALDO DO DIA"
df1 = df1[~df1["description"].str.contains("SALDO DO DIA", case=False, na=False)]


In [ ]:
# === LEITURA DO ARQUIVO DE CONFIGURAÇÃO ===
with open("config.yml", "r") as f:
    config = yaml.safe_load(f)

cards = config.get("cards", [])
fixed_income = config.get("fixed_income", [])
fixed_expenses = config.get("fixed_expenses", []) 

# === CONSTRUÇÃO DO DATAFRAME BASE ===
df = pd.DataFrame({"date": date_range})
df["inflow"] = 0.0
df["outflow"] = 0.0

# === ADICIONA UMA COLUNA PARA CADA CARTÃO ===
for card in cards:
    col = f"{card['bank'].capitalize()} - {card['name'].capitalize()} ({card['last_digits']})"
    df[col] = 0.0

# === CONVERTE DATAS PARA date (sem hora) ===
df["date"] = pd.to_datetime(df["date"]).dt.date
df1["date"] = pd.to_datetime(df1["date"]).dt.date

# === Agrupa extrato por data ===
extrato_por_dia = df1.groupby("date")[["inflow", "outflow"]].sum().reset_index()
extrato_por_dia["date"] = pd.to_datetime(extrato_por_dia["date"]).dt.date

# === Determina datas com extrato e última data real ===
dias_com_extrato = set(df1["date"].unique())
last_real_date = df1["date"].max()

# === APLICA RECEITAS FIXAS EM DATAS FUTURAS SEM EXTRATO ===
for income in fixed_income:
    income_day = income["day"]
    amount = income["amount"]
    df.loc[
        (df["date"].apply(lambda d: d.day == income_day and d > last_real_date and d not in dias_com_extrato)),
        "inflow"
    ] += amount

# === APLICA DESPESAS FIXAS EM DATAS FUTURAS SEM EXTRATO ===
for expense in fixed_expenses:
    expense_day = expense["day"]
    amount = expense["amount"]
    df.loc[
        (df["date"].apply(lambda d: d.day == expense_day and d > last_real_date and d not in dias_com_extrato)),
        "outflow"
    ] += amount

# === Merge com extrato para sobrescrever valores reais ===
df_merge = df.merge(extrato_por_dia, on="date", how="left", suffixes=("", "_real"))

# Substitui valores com extrato se disponíveis
df_merge["inflow"] = df_merge["inflow_real"].combine_first(df_merge["inflow"])
df_merge["outflow"] = df_merge["outflow_real"].combine_first(df_merge["outflow"])

# Zera valores antes da maior data do extrato quando não há entrada real
df_merge["inflow"] = df_merge.apply(
    lambda row: row["inflow"] if row["date"] > last_real_date or row["date"] in dias_com_extrato else 0,
    axis=1
)
df_merge["outflow"] = df_merge.apply(
    lambda row: row["outflow"] if row["date"] > last_real_date or row["date"] in dias_com_extrato else 0,
    axis=1
)

# Remove colunas temporárias
df_merge = df_merge.drop(columns=["inflow_real", "outflow_real"])

# Reordena colunas
ordered_columns = ["date", "inflow", "outflow"] + [
    col for col in df_merge.columns if col not in ["date", "inflow", "outflow"]
]
df = df_merge[ordered_columns]


In [139]:
# Recriar DataFrame com coluna balance zerada antes de gerar o Excel
df["balance"] = 0  # apenas placeholder para permitir exportação com estrutura completa

# Criar workbook
wb = Workbook()
ws = wb.active
ws.title = "Cashflow"

# Inserir DataFrame no worksheet
from openpyxl.utils.dataframe import dataframe_to_rows

for row in dataframe_to_rows(df, index=False, header=True):
    ws.append(row)

# Mapear colunas
header = [cell.value for cell in ws[1]]
col_idx = {col: idx + 1 for idx, col in enumerate(header)}

# Carregar config.yml com cartões e datas de vencimento
with open("config.yml", "r") as f:
    config = yaml.safe_load(f)

cards = config["cards"]
card_due_days = {
    f"{c['bank'].capitalize()} - {c['name'].capitalize()} ({c['last_digits']})": c["due_day"]
    for c in cards
}
card_colors = {
    f"{c['bank'].capitalize()} - {c['name'].capitalize()} ({c['last_digits']})": c["color"]
    for c in cards
}

# === Aplicar cores dos cartões ===
for card_name, colors in card_colors.items():
    if card_name in col_idx:
        col_letter = get_column_letter(col_idx[card_name])
        # Cabeçalho
        ws[f"{col_letter}1"].fill = PatternFill(start_color=colors[0][1:], end_color=colors[0][1:], fill_type="solid")
        for row in range(2, ws.max_row + 1):
            fill_color = colors[1] if row % 2 == 0 else colors[2]
            ws[f"{col_letter}{row}"].fill = PatternFill(start_color=fill_color[1:], end_color=fill_color[1:], fill_type="solid")

# === Aplicar fórmula na coluna balance ===
balance_col_letter = get_column_letter(col_idx["balance"])
inflow_letter = get_column_letter(col_idx["inflow"])
outflow_letter = get_column_letter(col_idx["outflow"])
date_letter = get_column_letter(col_idx["date"])

for row in range(2, ws.max_row + 1):
    inflow_cell = f"{inflow_letter}{row}"
    outflow_cell = f"{outflow_letter}{row}"
    prev_balance = f"{balance_col_letter}{row-1}" if row > 2 else "0"

    deductions = []
    for card, due_day in card_due_days.items():
        if card in col_idx:
            card_letter = get_column_letter(col_idx[card])
            deductions.append(f"IF(DAY({date_letter}{row})={due_day},{card_letter}{row},0)")

    deduction_expr = "-(" + "+".join(deductions) + ")" if deductions else ""
    formula = f"={prev_balance}+{inflow_cell}-{outflow_cell}{deduction_expr}"

    cell = ws[f"{balance_col_letter}{row}"]
    cell.value = formula
    cell.font = Font(bold=True)
    cell.alignment = Alignment(horizontal="center", vertical="center")

# === Aplicar formatação condicional na coluna balance ===
balance_range = f"{balance_col_letter}2:{balance_col_letter}{ws.max_row}"
color_scale = ColorScaleRule(
    start_type='num', start_value=-5000, start_color='FF0000',
    mid_type='num', mid_value=0, mid_color='FFFF00',
    end_type='num', end_value=20000, end_color='00FF00'
)
ws.conditional_formatting.add(balance_range, color_scale)

# === Centralizar todas as células ===
for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
    for cell in row:
        cell.alignment = Alignment(horizontal="center", vertical="center")

# === Salvar arquivo final formatado ===
final_xlsx_path = "final_cashflow_formatted.xlsx"
wb.save(final_xlsx_path)